# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession

session = SparkSession.builder.getOrCreate()
session

In [3]:
session.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

#### Passing other options to spark session:
    
    

In [4]:
session = SparkSession.builder.config('someoption.key', 'somevalue').getOrCreate()

Podemos ver los valores de las opciones de la sesión que tenemos creada así:

In [5]:
session.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1525863249715'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('someoption.key', 'somevalue'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.driver.host', '192.168.1.35'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '40639')]

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [6]:
import random

random.choice(['a', 'b', 'c'])

'a'

In [7]:
random.seed(42)

ids = range(12)

positions = [random.choice(['mechanic', 'sales', 'manager']) for id_ in ids]

positions

['manager',
 'mechanic',
 'mechanic',
 'manager',
 'sales',
 'mechanic',
 'mechanic',
 'mechanic',
 'manager',
 'mechanic',
 'manager',
 'manager']

In [8]:
rows = zip(ids, positions)
df = session.createDataFrame(rows)

df

DataFrame[_1: bigint, _2: string]

In [9]:
df.take(5)

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mechanic'),
 Row(_1=2, _2='mechanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales')]

In [10]:
df.show(5)

+---+--------+
| _1|      _2|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [11]:
from pyspark.sql import Row

help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <Row(name, age)>
 |  >>> 'name' 

In [12]:
rows = [Row(id=id_, position=position_) for id_, position_ in zip(ids, positions)]

rows

[Row(id=0, position='manager'),
 Row(id=1, position='mechanic'),
 Row(id=2, position='mechanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales'),
 Row(id=5, position='mechanic'),
 Row(id=6, position='mechanic'),
 Row(id=7, position='mechanic'),
 Row(id=8, position='manager'),
 Row(id=9, position='mechanic'),
 Row(id=10, position='manager'),
 Row(id=11, position='manager')]

In [13]:
df = session.createDataFrame(rows)

df.show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [14]:
session.createDataFrame(zip(ids, positions), schema=['id', 'position'])

DataFrame[id: bigint, position: string]

In [15]:
df.rdd

MapPartitionsRDD[29] at javaToPython at NativeMethodAccessorImpl.java:0

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [16]:
rdd = session.sparkContext.textFile('data/coupon150720.csv')

rdd

data/coupon150720.csv MapPartitionsRDD[31] at textFile at NativeMethodAccessorImpl.java:0

In [17]:
rdd.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [18]:
split_rdd = rdd.map(lambda line: line.split(','))

split_rdd.take(2)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0']]

In [19]:
new_df = session.createDataFrame(split_rdd)
new_df

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

### Inferring and specifying schemas

In [20]:
session.createDataFrame(zip(ids, positions), schema=['id', 'position'])

DataFrame[id: bigint, position: string]

In [21]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [22]:
from pyspark.sql import types

types.IntegerType()

IntegerType

In [23]:
fields = [types.StructField('id', types.IntegerType()), 
          types.StructField('position', types.StringType())]

my_schema = types.StructType(fields)

my_schema

StructType(List(StructField(id,IntegerType,true),StructField(position,StringType,true)))

In [24]:
session.createDataFrame(zip(ids, positions), schema=my_schema)

DataFrame[id: int, position: string]

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [25]:
df_from_csv = session.read.csv('data/coupon150720.csv')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [26]:
df_with_sql = session.sql('SELECT * FROM csv.`data/coupon150720.csv` where _c2="MAA"')

df_with_sql.show(2)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W|56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005305018|  1|MAA|BOM| 9W| 9W| 48.5|USD|  1|  O|   O|0460|150821|  OK|IAE0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 2 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [27]:
session.read.json
session.read.parquet


<bound method DataFrameReader.parquet of <pyspark.sql.readwriter.DataFrameReader object at 0x7f5196aa8080>>

### Basic operations with DataFrames

In [28]:
df.show(2)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
+---+--------+
only showing top 2 rows



In [29]:
df.take(2)

[Row(id=0, position='manager'), Row(id=1, position='mechanic')]

In [30]:
df.schema

StructType(List(StructField(id,LongType,true),StructField(position,StringType,true)))

In [31]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



### Filtering and selecting

Syntax inspired in SQL.

In [32]:
df.select('id')

DataFrame[id: bigint]

In [33]:
df.select(df['id'])

DataFrame[id: bigint]

If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [34]:
filtered = df.filter(df['id'] > 5)
filtered

DataFrame[id: bigint, position: string]

That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

In [35]:
filtered = df.filter('id' > 5)
filtered

TypeError: '>' not supported between instances of 'str' and 'int'

In [36]:
filtered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



`where` is exactly synonimous with `filter`

In [37]:
whered = df.where(df['id'] > 5)
whered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [38]:
df['id']

Column<b'id'>

In [39]:
df['id'] > 5

Column<b'(id > 5)'>

A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

In [46]:
col = df['id']
col.collect()

TypeError: 'Column' object is not callable

#### Exercise

Extract all employee ids which correspond to managers

In [41]:
df.where(df['position'] == 'manager').select('id').show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



In [42]:
df[df['position'] == 'manager'].select('id')

DataFrame[id: bigint]

In [43]:
df.select(df['position'] == 'manager').show()

+--------------------+
|(position = manager)|
+--------------------+
|                true|
|               false|
|               false|
|                true|
|               false|
|               false|
|               false|
|               false|
|                true|
|               false|
|                true|
|                true|
+--------------------+



In [47]:
df.select(col)

DataFrame[id: bigint]

### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we not assing to them. We need to create new RDDS with the appropriate columns.

In [48]:
df['anewcolumn'] = df['id'] * 100

TypeError: 'DataFrame' object does not support item assignment

In [49]:
df2 = df.withColumn('anewcolumn', df['id'] * 100)

df2

DataFrame[id: bigint, position: string, anewcolumn: bigint]

In [50]:
df

DataFrame[id: bigint, position: string]

In [51]:
df3 = df.select('id',
                'position',
                df['id'] * 100)

df3.show(4)

+---+--------+----------+
| id|position|(id * 100)|
+---+--------+----------+
|  0| manager|         0|
|  1|mechanic|       100|
|  2|mechanic|       200|
|  3| manager|       300|
+---+--------+----------+
only showing top 4 rows



### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allows us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [52]:
from pyspark.sql import functions

functions.sqrt(df['id'])

Column<b'SQRT(id)'>

In [53]:
df.select(functions.sqrt('id')).show(3)

+------------------+
|          SQRT(id)|
+------------------+
|               0.0|
|               1.0|
|1.4142135623730951|
+------------------+
only showing top 3 rows



In [54]:
df3 = df.select('id',
                'position',
                functions.sqrt('id'))

df3.show(4)

+---+--------+------------------+
| id|position|          SQRT(id)|
+---+--------+------------------+
|  0| manager|               0.0|
|  1|mechanic|               1.0|
|  2|mechanic|1.4142135623730951|
|  3| manager|1.7320508075688772|
+---+--------+------------------+
only showing top 4 rows



This errors out because 

```python
lambda col: math.log1p(col)
```

is not a udf: it doesn't know how to work with Column objects.

In [55]:
import math

math.log1p(0)

0.0

In [56]:
df3 = df.select('id',
                'position',
                lambda col: math.log1p(col))

df3.show(4)

TypeError: Invalid argument, not a string or column: <function <lambda> at 0x7f516d0cbf28> of type <class 'function'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [57]:
math.log1p(col)

TypeError: must be real number, not Column

But we can transform it into a udf:

In [58]:
udf_log1p = functions.udf(math.log1p)

In [59]:
udf_log1p(col)

Column<b'log1p(id)'>

In [60]:
df3 = df.select('id',
                'position',
                udf_log1p('id'))

df3.show(4)

+---+--------+------------------+
| id|position|         log1p(id)|
+---+--------+------------------+
|  0| manager|               0.0|
|  1|mechanic|0.6931471805599453|
|  2|mechanic|1.0986122886681096|
|  3| manager|1.3862943611198906|
+---+--------+------------------+
only showing top 4 rows



We can do the same with any function we dream up:

In [61]:
def odd_letters(word):
    return word[::2]
    
odd_letters('manager')

'mngr'

In [62]:
odd_udf = functions.udf(odd_letters)
odd_udf(col)

Column<b'odd_letters(id)'>

In [63]:
df.select(odd_udf('position')).show(5)

+---------------------+
|odd_letters(position)|
+---------------------+
|                 mngr|
|                 mcai|
|                 mcai|
|                 mngr|
|                  sls|
+---------------------+
only showing top 5 rows



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [64]:
df3 = df.select('id',
                'position',
                udf_log1p('id'))

df3.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- log1p(id): string (nullable = true)



In [65]:
udf_log1p_typed = functions.udf(math.log1p, types.FloatType())

In [66]:
df3 = df.select('id',
                'position',
                udf_log1p_typed('id'))

df3.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- log1p(id): float (nullable = true)



Think about this function: what is its return type?

In [67]:
# It's impossible to know the return type of this function, because it depends 
# on the types of its arguments, which we don't know.

functions.udf(lambda x: x + x)

<function __main__.<lambda>>

In [68]:
random_ab = functions.udf(lambda : random.choice(['a', 'b']))

# We can assign aliases to columns:
df.select('id',
          'position',
          random_ab().alias('a o b')).show(5)

+---+--------+-----+
| id|position|a o b|
+---+--------+-----+
|  0| manager|    b|
|  1|mechanic|    b|
|  2|mechanic|    b|
|  3| manager|    b|
|  4|   sales|    b|
+---+--------+-----+
only showing top 5 rows



#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [69]:
def salary_from(position):
    
    salaries = {'manager': 70000, 'sales' : 40000, 'mechanic' : 30000}
    
    return salaries.get(position)

salary_from('manager')

70000

In [70]:
salary_udf = functions.udf(salary_from)

If we have a column that is not the desired type, we can convert it with `cast`.

In [71]:
exercise_result = df.select('id',
                            'position',
                            salary_udf('position').alias('salary').cast(types.IntegerType()))

exercise_result.show(5)

+---+--------+------+
| id|position|salary|
+---+--------+------+
|  0| manager| 70000|
|  1|mechanic| 30000|
|  2|mechanic| 30000|
|  3| manager| 70000|
|  4|   sales| 40000|
+---+--------+------+
only showing top 5 rows



In [72]:
exercise_result.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- salary: integer (nullable = true)



### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [73]:
exercise_result.stat.corr('id', 'salary')

0.20742413513940744

In [74]:
exercise_result.stat.cov('id', 'salary')

15000.0

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [75]:
location_udf = functions.udf(lambda : random.choice(['Madrid', 'Barcelona']))

In [76]:
df4 = exercise_result.withColumn('location', location_udf()).cache()

In [77]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|Barcelona|
|  2|mechanic| 30000|Barcelona|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|Barcelona|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|Barcelona|
|  8| manager| 70000|Barcelona|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|Barcelona|
| 11| manager| 70000|Barcelona|
+---+--------+------+---------+



In [78]:
df4.crosstab('position', 'location').show()

+-----------------+---------+
|position_location|Barcelona|
+-----------------+---------+
|            sales|        1|
|          manager|        5|
|         mechanic|        6|
+-----------------+---------+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [79]:
df4.groupby('location')

In [80]:
df4.toPandas().groupby('location')

GroupedData has several aggregation functions defined:

In [81]:
df4.groupby('location').mean().show()

+---------+-------+-----------+
| location|avg(id)|avg(salary)|
+---------+-------+-----------+
|Barcelona|    5.5|    47500.0|
+---------+-------+-----------+



In [82]:
df4.groupby('location').mean('salary').show()

+---------+-----------+
| location|avg(salary)|
+---------+-----------+
|Barcelona|    47500.0|
+---------+-----------+



We can do several aggregations in a single step, with a number of different syntaxes:

In [83]:
stats = df4.groupby('location').agg(functions.mean('salary').alias('average'), 
                                    functions.stddev_pop('salary').alias('stddev')
                                   )

stats.show()

+---------+-------+-----------------+
| location|average|           stddev|
+---------+-------+-----------------+
|Barcelona|47500.0|19202.86436967152|
+---------+-------+-----------------+



In [84]:
stats = df4.groupby('location').agg({'salary': 'mean', 'id' : 'mean'}).show()

+---------+-------+-----------+
| location|avg(id)|avg(salary)|
+---------+-------+-----------+
|Barcelona|    5.5|    47500.0|
+---------+-------+-----------+



### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [85]:
random.seed(42)

data = list(zip([10, 11, 9, 10, 9],
                [5000, 10000, 2000, 0, 1000],
                [random.choice(['Madrid', 'Barcelona', 'Sevilla']) for _ in range(5)]))

data

[(10, 5000, 'Sevilla'),
 (11, 10000, 'Madrid'),
 (9, 2000, 'Madrid'),
 (10, 0, 'Sevilla'),
 (9, 1000, 'Barcelona')]

In [86]:
new_df = session.createDataFrame(data, schema=['id', 'bonus', 'location'])

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [87]:
df4.join(new_df).show()

AnalysisException: 'Detected cartesian product for INNER join between logical plans\nInMemoryRelation [id#13L, position#14, salary#407, location#439], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- *(1) Project [id#13L, position#14, cast(pythonUDF0#444 as int) AS salary#407, pythonUDF1#445 AS location#439]\n      +- BatchEvalPython [salary_from(position#14), <lambda>()], [id#13L, position#14, pythonUDF0#444, pythonUDF1#445]\n         +- Scan ExistingRDD[id#13L,position#14]\nand\nLogicalRDD [id#821L, bonus#822L, location#823], false\nJoin condition is missing or trivial.\nUse the CROSS JOIN syntax to allow cartesian products between these relations.;'

In [88]:
df4.join(new_df, on='id').show()

+---+--------+------+---------+-----+---------+
| id|position|salary| location|bonus| location|
+---+--------+------+---------+-----+---------+
| 10| manager| 70000|Barcelona| 5000|  Sevilla|
| 11| manager| 70000|Barcelona|10000|   Madrid|
|  9|mechanic| 30000|Barcelona| 2000|   Madrid|
| 10| manager| 70000|Barcelona|    0|  Sevilla|
|  9|mechanic| 30000|Barcelona| 1000|Barcelona|
+---+--------+------+---------+-----+---------+



In [89]:
df4.join(new_df, on='id', how='left').show()

+---+--------+------+---------+-----+---------+
| id|position|salary| location|bonus| location|
+---+--------+------+---------+-----+---------+
|  0| manager| 70000|Barcelona| null|     null|
|  7|mechanic| 30000|Barcelona| null|     null|
|  6|mechanic| 30000|Barcelona| null|     null|
|  9|mechanic| 30000|Barcelona| 2000|   Madrid|
|  9|mechanic| 30000|Barcelona| 1000|Barcelona|
|  5|mechanic| 30000|Barcelona| null|     null|
|  1|mechanic| 30000|Barcelona| null|     null|
| 10| manager| 70000|Barcelona| 5000|  Sevilla|
| 10| manager| 70000|Barcelona|    0|  Sevilla|
|  3| manager| 70000|Barcelona| null|     null|
|  8| manager| 70000|Barcelona| null|     null|
| 11| manager| 70000|Barcelona|10000|   Madrid|
|  2|mechanic| 30000|Barcelona| null|     null|
|  4|   sales| 40000|Barcelona| null|     null|
+---+--------+------+---------+-----+---------+



In [90]:
with_bonuses = df4.join(new_df, on=['id', 'location'], how='left')
with_bonuses.show()

+---+---------+--------+------+-----+
| id| location|position|salary|bonus|
+---+---------+--------+------+-----+
|  3|Barcelona| manager| 70000| null|
|  5|Barcelona|mechanic| 30000| null|
|  7|Barcelona|mechanic| 30000| null|
|  0|Barcelona| manager| 70000| null|
|  6|Barcelona|mechanic| 30000| null|
|  9|Barcelona|mechanic| 30000| 1000|
| 11|Barcelona| manager| 70000| null|
|  2|Barcelona|mechanic| 30000| null|
| 10|Barcelona| manager| 70000| null|
|  8|Barcelona| manager| 70000| null|
|  1|Barcelona|mechanic| 30000| null|
|  4|Barcelona|   sales| 40000| null|
+---+---------+--------+------+-----+



In [91]:
type(with_bonuses)

pyspark.sql.dataframe.DataFrame

#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

In [92]:
sc.uiWebUrl

'http://192.168.1.35:4043'

In [93]:
from pyspark.storagelevel import StorageLevel

help(StorageLevel)

Help on class StorageLevel in module pyspark.storagelevel:

class StorageLevel(builtins.object)
 |  Flags for controlling the storage of an RDD. Each StorageLevel records whether to use memory,
 |  whether to drop the RDD to disk if it falls out of memory, whether to keep the data in memory
 |  in a JAVA-specific serialized format, and whether to replicate the RDD partitions on multiple
 |  nodes. Also contains static constants for some commonly used storage levels, MEMORY_ONLY.
 |  Since the data is always serialized on the Python side, all the constants use the serialized
 |  formats.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, useDisk, useMemory, useOffHeap, deserialized, replication=1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  

In [94]:
df4.storageLevel

StorageLevel(True, True, False, True, 1)

In [95]:
df4.persist(StorageLevel.MEMORY_ONLY)

DataFrame[id: bigint, position: string, salary: int, location: string]

In [96]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|Barcelona|
|  2|mechanic| 30000|Barcelona|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|Barcelona|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|Barcelona|
|  8| manager| 70000|Barcelona|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|Barcelona|
| 11| manager| 70000|Barcelona|
+---+--------+------+---------+



We can erase it with `unpersist`

In [97]:
df4.unpersist()

DataFrame[id: bigint, position: string, salary: int, location: string]

In [98]:
df4.persist(StorageLevel.MEMORY_AND_DISK_SER_2)

DataFrame[id: bigint, position: string, salary: int, location: string]

In [99]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|Barcelona|
|  2|mechanic| 30000|Barcelona|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|Barcelona|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|Barcelona|
|  8| manager| 70000|Barcelona|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|Barcelona|
| 11| manager| 70000|Barcelona|
+---+--------+------+---------+



#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

In [100]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|Barcelona|
|  2|mechanic| 30000|Barcelona|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|Barcelona|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|Barcelona|
|  8| manager| 70000|Barcelona|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|Barcelona|
| 11| manager| 70000|Barcelona|
+---+--------+------+---------+



1) Calculate the mean and std of salary for each location

In [101]:
stats = df4.groupby('location').agg(
    functions.mean('salary').alias('avg_salary'),
    functions.stddev_pop('salary').alias('std_salary'))

stats.show()

+---------+----------+-----------------+
| location|avg_salary|       std_salary|
+---------+----------+-----------------+
|Barcelona|   47500.0|19202.86436967152|
+---------+----------+-----------------+



2) Annotate each employee with the stats corresponding to their location

In [102]:
df5 = df4.join(stats, on='location', how='left')

df5.show()

+---------+---+--------+------+----------+-----------------+
| location| id|position|salary|avg_salary|       std_salary|
+---------+---+--------+------+----------+-----------------+
|Barcelona|  0| manager| 70000|   47500.0|19202.86436967152|
|Barcelona|  1|mechanic| 30000|   47500.0|19202.86436967152|
|Barcelona|  2|mechanic| 30000|   47500.0|19202.86436967152|
|Barcelona|  3| manager| 70000|   47500.0|19202.86436967152|
|Barcelona|  4|   sales| 40000|   47500.0|19202.86436967152|
|Barcelona|  5|mechanic| 30000|   47500.0|19202.86436967152|
|Barcelona|  6|mechanic| 30000|   47500.0|19202.86436967152|
|Barcelona|  7|mechanic| 30000|   47500.0|19202.86436967152|
|Barcelona|  8| manager| 70000|   47500.0|19202.86436967152|
|Barcelona|  9|mechanic| 30000|   47500.0|19202.86436967152|
|Barcelona| 10| manager| 70000|   47500.0|19202.86436967152|
|Barcelona| 11| manager| 70000|   47500.0|19202.86436967152|
+---------+---+--------+------+----------+-----------------+



3) Calculate the z-score

In [103]:
result = df5.select('id',
                    'position',
                    'location',
                    'salary',
                    ((df5['salary'] - df5['avg_salary']) / df5['std_salary']).alias('z-score'))

result.show()

+---+--------+---------+------+--------------------+
| id|position| location|salary|             z-score|
+---+--------+---------+------+--------------------+
|  0| manager|Barcelona| 70000|   1.171700198827415|
|  1|mechanic|Barcelona| 30000| -0.9113223768657671|
|  2|mechanic|Barcelona| 30000| -0.9113223768657671|
|  3| manager|Barcelona| 70000|   1.171700198827415|
|  4|   sales|Barcelona| 40000|-0.39056673294247163|
|  5|mechanic|Barcelona| 30000| -0.9113223768657671|
|  6|mechanic|Barcelona| 30000| -0.9113223768657671|
|  7|mechanic|Barcelona| 30000| -0.9113223768657671|
|  8| manager|Barcelona| 70000|   1.171700198827415|
|  9|mechanic|Barcelona| 30000| -0.9113223768657671|
| 10| manager|Barcelona| 70000|   1.171700198827415|
| 11| manager|Barcelona| 70000|   1.171700198827415|
+---+--------+---------+------+--------------------+



In [104]:
result.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- location: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- z-score: double (nullable = true)



Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

In [105]:
df4.join(stats, on=df4['location']==stats['location'], how='left')

DataFrame[id: bigint, position: string, salary: int, location: string, location: string, avg_salary: double, std_salary: double]

### Handling null values

In [106]:
the_nulls = session.createDataFrame([[101, 'manager', 1200000, None],
                                      [None, None, 1500000, 'Haiti']])

the_nulls.show()

+----+-------+-------+-----+
|  _1|     _2|     _3|   _4|
+----+-------+-------+-----+
| 101|manager|1200000| null|
|null|   null|1500000|Haiti|
+----+-------+-------+-----+



In [107]:
df6 =  df4.union(the_nulls)

df6.show()

+----+--------+-------+---------+
|  id|position| salary| location|
+----+--------+-------+---------+
|   0| manager|  70000|Barcelona|
|   1|mechanic|  30000|Barcelona|
|   2|mechanic|  30000|Barcelona|
|   3| manager|  70000|Barcelona|
|   4|   sales|  40000|Barcelona|
|   5|mechanic|  30000|Barcelona|
|   6|mechanic|  30000|Barcelona|
|   7|mechanic|  30000|Barcelona|
|   8| manager|  70000|Barcelona|
|   9|mechanic|  30000|Barcelona|
|  10| manager|  70000|Barcelona|
|  11| manager|  70000|Barcelona|
| 101| manager|1200000|     null|
|null|    null|1500000|    Haiti|
+----+--------+-------+---------+



In [108]:
df6.dropna(subset='location').show()

+----+--------+-------+---------+
|  id|position| salary| location|
+----+--------+-------+---------+
|   0| manager|  70000|Barcelona|
|   1|mechanic|  30000|Barcelona|
|   2|mechanic|  30000|Barcelona|
|   3| manager|  70000|Barcelona|
|   4|   sales|  40000|Barcelona|
|   5|mechanic|  30000|Barcelona|
|   6|mechanic|  30000|Barcelona|
|   7|mechanic|  30000|Barcelona|
|   8| manager|  70000|Barcelona|
|   9|mechanic|  30000|Barcelona|
|  10| manager|  70000|Barcelona|
|  11| manager|  70000|Barcelona|
|null|    null|1500000|    Haiti|
+----+--------+-------+---------+



In [109]:
df6.dropna(thresh=3).show()

+---+--------+-------+---------+
| id|position| salary| location|
+---+--------+-------+---------+
|  0| manager|  70000|Barcelona|
|  1|mechanic|  30000|Barcelona|
|  2|mechanic|  30000|Barcelona|
|  3| manager|  70000|Barcelona|
|  4|   sales|  40000|Barcelona|
|  5|mechanic|  30000|Barcelona|
|  6|mechanic|  30000|Barcelona|
|  7|mechanic|  30000|Barcelona|
|  8| manager|  70000|Barcelona|
|  9|mechanic|  30000|Barcelona|
| 10| manager|  70000|Barcelona|
| 11| manager|  70000|Barcelona|
|101| manager|1200000|     null|
+---+--------+-------+---------+



In [110]:
df6.fillna('unknown').show()

+----+--------+-------+---------+
|  id|position| salary| location|
+----+--------+-------+---------+
|   0| manager|  70000|Barcelona|
|   1|mechanic|  30000|Barcelona|
|   2|mechanic|  30000|Barcelona|
|   3| manager|  70000|Barcelona|
|   4|   sales|  40000|Barcelona|
|   5|mechanic|  30000|Barcelona|
|   6|mechanic|  30000|Barcelona|
|   7|mechanic|  30000|Barcelona|
|   8| manager|  70000|Barcelona|
|   9|mechanic|  30000|Barcelona|
|  10| manager|  70000|Barcelona|
|  11| manager|  70000|Barcelona|
| 101| manager|1200000|  unknown|
|null| unknown|1500000|    Haiti|
+----+--------+-------+---------+



In [111]:
df6.fillna('unknown', subset='location').show()

+----+--------+-------+---------+
|  id|position| salary| location|
+----+--------+-------+---------+
|   0| manager|  70000|Barcelona|
|   1|mechanic|  30000|Barcelona|
|   2|mechanic|  30000|Barcelona|
|   3| manager|  70000|Barcelona|
|   4|   sales|  40000|Barcelona|
|   5|mechanic|  30000|Barcelona|
|   6|mechanic|  30000|Barcelona|
|   7|mechanic|  30000|Barcelona|
|   8| manager|  70000|Barcelona|
|   9|mechanic|  30000|Barcelona|
|  10| manager|  70000|Barcelona|
|  11| manager|  70000|Barcelona|
| 101| manager|1200000|  unknown|
|null|    null|1500000|    Haiti|
+----+--------+-------+---------+



## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

In [112]:
df6.registerTempTable('df6_table')

session.sql('SELECT * FROM df6_table WHERE location IS NOT NULL AND position="manager"').show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  3| manager| 70000|Barcelona|
|  8| manager| 70000|Barcelona|
| 10| manager| 70000|Barcelona|
| 11| manager| 70000|Barcelona|
+---+--------+------+---------+



Once registered, we can perform queries as complex as we want.

In [113]:
session.sql("""SELECT location, 
                      avg(salary) as avg_salary, 
                      stddev_pop(salary) AS std_salary 
               FROM df6_table 
               GROUP BY location""")

DataFrame[location: string, avg_salary: double, std_salary: double]

In [114]:
df5.show(2)

+---------+---+--------+------+----------+-----------------+
| location| id|position|salary|avg_salary|       std_salary|
+---------+---+--------+------+----------+-----------------+
|Barcelona|  0| manager| 70000|   47500.0|19202.86436967152|
|Barcelona|  1|mechanic| 30000|   47500.0|19202.86436967152|
+---------+---+--------+------+----------+-----------------+
only showing top 2 rows



In [115]:
def zscore(value, mean, stddev):
    return (value - mean) / stddev

zscore_udf = functions.udf(zscore)

df5.select('id',
           zscore_udf('salary', 'avg_salary', 'std_salary').alias('z-score')
          ).show()

+---+--------------------+
| id|             z-score|
+---+--------------------+
|  0|   1.171700198827415|
|  1| -0.9113223768657671|
|  2| -0.9113223768657671|
|  3|   1.171700198827415|
|  4|-0.39056673294247163|
|  5| -0.9113223768657671|
|  6| -0.9113223768657671|
|  7| -0.9113223768657671|
|  8|   1.171700198827415|
|  9| -0.9113223768657671|
| 10|   1.171700198827415|
| 11|   1.171700198827415|
+---+--------------------+



In [116]:
df5.registerTempTable('df5_table')
session.udf.register('tocoto', zscore)

<function __main__.zscore>

In [117]:
session.sql('SELECT id, tocoto(salary, avg_salary, std_salary) AS z_score FROM df5_table').show()

+---+--------------------+
| id|             z_score|
+---+--------------------+
|  0|   1.171700198827415|
|  1| -0.9113223768657671|
|  2| -0.9113223768657671|
|  3|   1.171700198827415|
|  4|-0.39056673294247163|
|  5| -0.9113223768657671|
|  6| -0.9113223768657671|
|  7| -0.9113223768657671|
|  8|   1.171700198827415|
|  9| -0.9113223768657671|
| 10|   1.171700198827415|
| 11|   1.171700198827415|
+---+--------------------+



#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

In [118]:
pd_df = df5.toPandas()
pd_df.head(5)

,location,id,position,salary,avg_salary,std_salary
0,Barcelona,0,manager,70000,47500.0,19202.86437
1,Barcelona,1,mechanic,30000,47500.0,19202.86437
2,Barcelona,2,mechanic,30000,47500.0,19202.86437
3,Barcelona,3,manager,70000,47500.0,19202.86437
4,Barcelona,4,sales,40000,47500.0,19202.86437


In [119]:
type(pd_df)

pandas.core.frame.DataFrame

In [120]:
session.createDataFrame(pd_df)

DataFrame[location: string, id: bigint, position: string, salary: bigint, avg_salary: double, std_salary: double]

## Writing out


In [122]:
df5.write.csv('df5')

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

In [123]:
raw_coupons = session.read.csv('data/coupon150720.csv')
raw_coupons

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string]

In [124]:
raw_coupons.show(10)

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA|  9W|  9W|  60.0|USD|  1|   H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE|  9W|  9W|152.46|USD|  1|   V|   V|0501|150803|  OK|INA0|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IAE0|
|79062006192650|  2|BOM|RPR|  9W|  9W|  68.5|USD|  1|   H|   H|0377|150721|  OK|IAE0|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|   V|2839|150801|  OK|INA0|
|79062005836987|  1|ATL|LGA|  AA|  AA|  28.3|USD|  1| 

In [125]:
raw_coupons.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)



1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

In [126]:
coupons = raw_coupons.select(raw_coupons['_c0'].alias('tkt_number'),
                             raw_coupons['_c1'].alias('cpn_number').cast(types.IntegerType()),
                             raw_coupons['_c2'].alias('origin'),
                             raw_coupons['_c3'].alias('destination'),
                             raw_coupons['_c4'].alias('airline'),
                             raw_coupons['_c6'].alias('amount').cast(types.FloatType())
                            )
coupons.show(5)
coupons.printSchema()

+--------------+----------+------+-----------+-------+------+
|    tkt_number|cpn_number|origin|destination|airline|amount|
+--------------+----------+------+-----------+-------+------+
|79062005698500|         1|   MAA|        AUH|     9W| 56.79|
|79062005698500|         2|   AUH|        CDG|     9W| 84.34|
|79062005924069|         1|   CJB|        MAA|     9W|  60.0|
|79065668570385|         1|   DEL|        DXB|     9W|160.63|
|79065668737021|         1|   AUH|        IXE|     9W|152.46|
+--------------+----------+------+-----------+-------+------+
only showing top 5 rows

root
 |-- tkt_number: string (nullable = true)
 |-- cpn_number: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- amount: float (nullable = true)



In [127]:
raw_coupons.registerTempTable('raw_coupons')

coupons = session.sql('''SELECT 
                            CAST(_c0 AS LONG) AS tkt_number,
                            CAST(_c1 AS INT) AS cpn_number,
                            _c2 AS origin,
                            _c3 AS destination,
                            _c4 AS airline,
                            CAST(_c6 AS FLOAT) AS amount
                         FROM raw_coupons''')

coupons.show(5)
coupons.printSchema()

+--------------+----------+------+-----------+-------+------+
|    tkt_number|cpn_number|origin|destination|airline|amount|
+--------------+----------+------+-----------+-------+------+
|79062005698500|         1|   MAA|        AUH|     9W| 56.79|
|79062005698500|         2|   AUH|        CDG|     9W| 84.34|
|79062005924069|         1|   CJB|        MAA|     9W|  60.0|
|79065668570385|         1|   DEL|        DXB|     9W|160.63|
|79065668737021|         1|   AUH|        IXE|     9W|152.46|
+--------------+----------+------+-----------+-------+------+
only showing top 5 rows

root
 |-- tkt_number: long (nullable = true)
 |-- cpn_number: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- amount: float (nullable = true)



2) Total ticket amounts per origin

In [128]:
coupons[coupons['destination'] == 'MAD'].groupby('origin').sum('amount').show(5)

+------+------------------+
|origin|       sum(amount)|
+------+------------------+
|   PMI| 40547.17005729675|
|   YUL|284.44000244140625|
|   HEL| 8195.760055541992|
|   SXB| 264.4599914550781|
|   UIO| 8547.599964141846|
+------+------------------+
only showing top 5 rows



3) Top 10 Airlines by average amount



In [129]:
airlines = coupons[coupons['destination'] == 'MAD'].groupby('airline').mean('amount')
airlines.show(5)

+-------+------------------+
|airline|       avg(amount)|
+-------+------------------+
|     PC| 64.16499996185303|
|     AZ| 68.80148202401621|
|     UA| 291.9203567504883|
|     RO|150.18356997626168|
|     LA|379.95370341954606|
+-------+------------------+
only showing top 5 rows



In [130]:
airlines.sort('avg(amount)', ascending=False).show(10)

+-------+------------------+
|airline|       avg(amount)|
+-------+------------------+
|     V0| 5418.098665364583|
|     AC|  740.619985961914|
|     KE| 688.5261500431941|
|     SV|  553.174259916265|
|     OB| 535.5044420030382|
|     AR| 513.5304808843704|
|     AV| 450.1950941518613|
|     AM| 440.7342111687911|
|     C2| 397.8699951171875|
|     LA|379.95370341954606|
+-------+------------------+
only showing top 10 rows



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

